# 自製智能中文選字系統  (2)

In [1]:
import sys
sys.version

'3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]'

確認版本為 python3

## 資料前處理

In [2]:
import re

In [3]:
def prepocess_line(line):
    chinese_pattern = r'[\u4E00-\u9FFF]+'
    segments = re.findall(chinese_pattern, line, flags=re.UNICODE)
    return segments

In [5]:
segments = []
with open('./wiki_zh_small.txt', encoding = 'utf8') as fr:
    for line in fr.readlines():
        segments += prepocess_line(line)

## 斷詞

In [6]:
!pip install jieba

  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314477 sha256=2679381f3b40e13012ad6cad4d714a7467786fa07bad1323c9161eff55ba37a3
  Stored in directory: c:\users\yz830\appdata\local\pip\cache\wheels\24\aa\17\5bc7c72e9a37990a9620cc3aad0acad1564dcff6dbc2359de3
Successfully built jieba


In [7]:
import jieba

In [8]:
list(jieba.cut_for_search(segments[6001]))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\yz830\AppData\Local\Temp\jieba.cache
Loading model cost 0.675 seconds.
Prefix dict has been built successfully.


['所以', '僅用', '於', '還原', '一些', '貴重', '的', '化合', '化合物']

In [9]:
cut_segments = []
for seg in segments:
    # 使用結巴斷詞的 cut_for_search
    cut_segments += list(jieba.cut_for_search(seg))

## 使用斷詞的結果來作Ngram

In [10]:
from collections import Counter

class Counters:
    def __init__(self, n):
        self.n = n
        self.counters = [Counter() for _ in range(n + 1)]

    def fit(self, segments):
        for i in range(1, 1 + self.n):
            for segment in segments:
                self.counters[i] += Counter(self._skip(segment, i))

        base_count = sum(dict(self.counters[1]).values())
        self.counters[0] = Counter({'': base_count})

    def __getitem__(self, k):
        return self.counters[k]

    def _skip(self, segment, n):
        assert n > 0
        if len(segment) < n:
            return []
        shift = n - 1
        for i in range(len(segment) - shift):
            yield segment[i:i+shift+1]

In [11]:
counters = Counters(n=5)
counters.fit(cut_segments)

In [12]:
class Ngram:
    def __init__(self, n, counters):
        assert n <= counters.n
        self.n = n
        self.major_counter = counters[n]
        self.minor_counter = counters[n-1]

    def predict_proba(self, prefix='', top_k=5):
        assert len(prefix) >= self.n - 1

        reference = prefix[-(self.n - 1):] if self.n > 1 else ''
        count_referecne = self.minor_counter[reference]
        probs = []
        for key, count in dict(self.major_counter).items():
            if key.startswith(reference):
                prob = count / count_referecne
                probs.append((prob, key[-1]))
        sorted_probs = sorted(probs, reverse=True)
        return sorted_probs[:top_k] if top_k > 0 else sorted_probs

    def get_proba_dict(self, prefix=''):
        return {word: prob for prob, word in self.predict_proba(prefix, top_k=-1)}

In [13]:
ngrams = [Ngram(i, counters) for i in range(1, 6)]

## 使用Smoothing of Language Models來建立第二版選字系統

In [71]:
class ChineseWordRecommenderV2:
    def __init__(self, ngrams):
        self.ngrams = ngrams
    
    def predict_proba(self, prefix='', top_k=5):
        interpolation_lambda = 0.99
        proba_dicts = [ngram.get_proba_dict(prefix) for ngram in ngrams[:len(prefix)+1]]

        probas = []
        # 在此你可以選擇兩種 Smoothing of Language Models 的方法：
        # Back-off Smoothing 或 Interpolation Smoothing
        # 如果你選擇 Interpolation Smoothing，你可以運用已經準備好的輔助函式
        # _get_interpolation_proba 來達到此目的
        # YOUR CODE HERE
        words = proba_dicts[0].keys()
        for word in words:
            proba = self._get_interpolation_proba(word, proba_dicts, interpolation_lambda)
            probas.append((proba, word))
        sorted_probas = sorted(probas, reverse=True)
        # END YOUR CODE
        return sorted_probas[:top_k] if top_k > 0 else sorted_probas
    
    def _get_interpolation_proba(self, word, proba_dicts, interp_lambda, idx=None):
        if idx is None:
            idx = len(proba_dicts) - 1
        if idx == 0:
            return proba_dicts[idx].get(word, 0.)
        return interp_lambda * proba_dicts[idx].get(word, 0.) + \
               (1 - interp_lambda) * self._get_interpolation_proba(word, proba_dicts, interp_lambda, idx=idx-1)

In [72]:
model = ChineseWordRecommenderV2(ngrams)

In [73]:
probs = model.predict_proba('法律之', top_k=10)
probs

[(1.4220076768015646e-05, '一'),
 (9.707570320760328e-06, '間'),
 (5.28197056876624e-06, '前'),
 (3.728881726935905e-06, '外'),
 (2.3326629485747186e-06, '後'),
 (1.8654770669406341e-06, '下'),
 (1.4083935326765796e-06, '中'),
 (1.0881637677730107e-06, '意'),
 (1.0130499336554234e-06, '上'),
 (8.551614883949419e-07, '戰')]

## Demo

In [28]:
!pip install -U pip
!pip install -q ipywidgets

In [74]:
import ipywidgets as widgets

text = widgets.Textarea()
label = widgets.Label()
display(label, text)

def func(change):
    probs = model.predict_proba(change.new, top_k=10)
    label.value = ' ' + '\t'.join([word for prob, word in probs])

text.observe(func, names='value')

Label(value='')

Textarea(value='')